First, make some essential imports and then have a look at the data directory. The cell below gives a peek into what is in the BreakHis dataset. The goal is to initially create a pandas dataframe with two columns: images and labels. In this dataset, the file name itself contain the labels, so we will use that information.

In [1]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
        # print(os.path.join(dirname, filename))

import time
import datetime
timestamp_exec_start = time.time()

Just the filenames are enough to get class information of the images. There are four classes for benign, and four for malignant. Ignoring the magnification, there is a total of 8 classes. The next cell simply makes a list of all the files that are images. 

In [2]:
files = []
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if (filename[-3:] == 'png'):
            files.append(os.path.join(dirname, filename))
print("Processed ",len(files),"files")

Processed  7909 files


Next, we build the class labels from the file names and then build up a dataframe. Based on the value of the "REDUCED_CLASSES" flag, we either take 8 classes, or we take 2 classes. 
- B  = Benign
    * A = Adenosis
    * F = Fibroadenoma
    * TA = Tubular Adenoma
    * PT = Phyllodes Tumor
- M  = Malignant
    * DC = Ductal Carcinoma
    * LC = Lobular Carcinoma
    * MC = Mucinous Carcinoma (Colloid)
    * PC = Papillary Carcinoma
 

In [3]:
labels_dict = {"B_A-":0,"B_F-":1,"B_TA":2,"B_PT":3,"M_DC":4,"M_LC":5,"M_MC":6,"M_PC":7}  # for 8 class problem
labels_dict_simple = {"B":0,"M":1}                                                       # for 2 class problem
REDUCED_CLASSES = True

X = []
Y = []
for f in files:
    x = f.split("/") # break up the path
    x = x[-1:][0]    # extract the file name
    X.append(str(f))
    if REDUCED_CLASSES:
        Y.append(int(labels_dict_simple[x[4]]))
    else:
        Y.append(int(labels_dict[x[4:8]]))

data = {"images":X,"labels":Y}
images_df = pd.DataFrame(data, columns = ['images','labels'])
images_df.groupby("labels")["labels"].count()

labels
0    2480
1    5429
Name: labels, dtype: int64

Import Pytorch Stuff

In [4]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset


In [5]:
train, val = train_test_split(images_df, stratify=images_df.labels, test_size=0.2)
len(train), len(val)

(6327, 1582)

In [6]:
class MyDataset(Dataset):
    def __init__(self, df_data,transform=None):
        super().__init__()
        self.df = df_data.values
        
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path,label = self.df[index]
        
        image = cv2.imread(img_path)
        image = cv2.resize(image, (224,224))
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [7]:
## Parameters for model

# Hyper parameters
num_epochs = 50
num_classes = 8
batch_size = 16
learning_rate = 0.0002

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [8]:
# Utility function for saving model
# During training, the loss values are stored in a list.
# We check the last two values to see if the loss has reduced.
def save_checkpoint(state, loss):
    global best_loss
    """Save checkpoint if a new best is achieved"""
    if best_loss>=loss:        
        print ("=> Loss reduced by:\t",best_loss - loss)
        print("   Saving model state")
        torch.save(state, "state_dict.dct")  # save checkpoint
        best_loss = loss

In [9]:
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20), 
                                  transforms.Resize(224, interpolation = 2),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

trans_valid = transforms.Compose([transforms.ToPILImage(),                    
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.Resize(224, interpolation = 2),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

dataset_train = MyDataset(df_data=train, transform=trans_train)
dataset_valid = MyDataset(df_data=val,transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=False, num_workers=0)

Now, we create the model and train it.

In [10]:
import torch.nn as nn
from torchvision import models

def convrelu(in_channels, out_channels, kernel, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel, padding=padding),
        nn.ReLU(inplace=True),
    )


class ResNetUNet(nn.Module):
    def __init__(self, n_class):
        super().__init__()

        self.base_model = models.resnet18(pretrained=True)
        self.base_layers = list(self.base_model.children())

        self.layer0 = nn.Sequential(*self.base_layers[:3]) # size=(N, 64, x.H/2, x.W/2)
        self.layer0_1x1 = convrelu(64, 64, 1, 0)
        self.layer1 = nn.Sequential(*self.base_layers[3:5]) # size=(N, 64, x.H/4, x.W/4)
        self.layer1_1x1 = convrelu(64, 64, 1, 0)
        self.layer2 = self.base_layers[5]  # size=(N, 128, x.H/8, x.W/8)
        self.layer2_1x1 = convrelu(128, 128, 1, 0)
        self.layer3 = self.base_layers[6]  # size=(N, 256, x.H/16, x.W/16)
        self.layer3_1x1 = convrelu(256, 256, 1, 0)
        self.layer4 = self.base_layers[7]  # size=(N, 512, x.H/32, x.W/32)
        self.layer4_1x1 = convrelu(512, 512, 1, 0)

        self.upsample = nn.Upsample(scale_factor=2, mode='bicubic', align_corners=True) # this is now not being used. Keeping it for testing only.
        
        # o = s*(n-1) + f - 2*p
        self.upsample_4_3 = nn.ConvTranspose2d(in_channels = 512, out_channels = 512, kernel_size = 4, stride = 2, padding = 1)  # 7 -> 14
        self.upsample_3_2 = nn.ConvTranspose2d(in_channels = 512, out_channels = 512, kernel_size = 4, stride = 2, padding = 1) # 14 -> 28
        self.upsample_2_1 = nn.ConvTranspose2d(in_channels = 256, out_channels = 256, kernel_size = 4, stride = 2, padding = 1) # 28 -> 56
        self.upsample_1_0 = nn.ConvTranspose2d(in_channels = 256, out_channels = 256, kernel_size = 4, stride = 2, padding = 1) # 56 -> 112
        self.upsample_0_f = nn.ConvTranspose2d(in_channels = 128, out_channels = 128, kernel_size = 4, stride = 2, padding = 1) # 112 -> 224
        
        self.conv_up3 = convrelu(256 + 512, 512, 3, 1)
        self.conv_up2 = convrelu(128 + 512, 256, 3, 1)
        self.conv_up1 = convrelu(64 + 256, 256, 3, 1)
        self.conv_up0 = convrelu(64 + 256, 128, 3, 1)

        self.conv_original_size0 = convrelu(3, 64, 3, 1)
        self.conv_original_size1 = convrelu(64, 64, 3, 1)
        self.conv_original_size2 = convrelu(64 + 128, 64, 3, 1)

        self.conv_last = nn.Conv2d(64, n_class, 1)
        
        self.avg = nn.AvgPool2d(8) # 8x2x28x28
        self.fc = nn.Linear(n_class*28*28,2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input):
        x_original = self.conv_original_size0(input)
        x_original = self.conv_original_size1(x_original)

        layer0 = self.layer0(input)
        layer1 = self.layer1(layer0)
        layer2 = self.layer2(layer1)
        layer3 = self.layer3(layer2)
        layer4 = self.layer4(layer3)

        layer4 = self.layer4_1x1(layer4)
        x = self.upsample(layer4)
        
        layer3 = self.layer3_1x1(layer3)
        x = torch.cat([x, layer3], dim=1)
        x = self.conv_up3(x)
        x = self.upsample(x)
        layer2 = self.layer2_1x1(layer2)
        x = torch.cat([x, layer2], dim=1)
        x = self.conv_up2(x)
        x = self.upsample(x)
        
        layer1 = self.layer1_1x1(layer1)
        x = torch.cat([x, layer1], dim=1)
        x = self.conv_up1(x)
        x = self.upsample(x)  #       
        layer0 = self.layer0_1x1(layer0)
        x = torch.cat([x, layer0], dim=1)
        x = self.conv_up0(x)
        x = self.upsample(x)
        x = torch.cat([x, x_original], dim=1)
        x = self.conv_original_size2(x)

        out = self.conv_last(x)  
        out = self.avg(out)
        out = out.view(-1,num_classes*28*28)
        out = self.fc(out)
        # out = self.softmax(out)
        
        return out

In [11]:
model = ResNetUNet(num_classes).to(device)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /tmp/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 95.9MB/s]


In [12]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate)

In [13]:
import os
import urllib.request
url = "https://onedrive.live.com/download?cid=E5569BBAB912B6A5&resid=E5569BBAB912B6A5%21484258&authkey=AAyx18QtkAxB4D8"
file_name = "download.dct"
print("Downloading pre-trained model state...")
urllib.request.urlretrieve(url, file_name)
print("Done!")
model.load_state_dict(torch.load(file_name))
import os.path
if os.path.exists(file_name):
    try:
        model.load_state_dict(torch.load(file_name))
        print("Downloaded model state loaded successfully.")
    except:
        print("Error loading state dict.")
        

Done!
Downloaded model state loaded successfully.


At this point, we download a pre-trained model that was trained during the previous sesstions. OneDrive allows direct downloads. Simply open the file in the browser in OneDrive and copy the link in the **embed** code. In the URL string, replace "embed" with "download" and you are good to go. Every time you upload a file, simply change the URL.


In [14]:
# Train the model
timestamp_train_start = time.time()


loss_hist = []
best_loss = 9
total_step = len(loader_train)

for epoch in range(num_epochs):
    timestamp_epoch_start = time.time()
    print("Epoch ", epoch+1," started...")
    for i, (images, labels) in enumerate(loader_train):
        images = images.to(device)
        labels = labels.to(device)
                
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
       
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))    
             # Checkpointing
            loss_hist.append(float(loss.item())) #add current loss value.
            save_checkpoint(model.state_dict(),float(loss.item()))
                    
    timestamp_epoch_end = time.time()
    print("Epoch done in ",str(datetime.timedelta(seconds=(timestamp_epoch_end - timestamp_epoch_start))))
    
timestamp_train_end = time.time()
print("Training done in ",str(datetime.timedelta(seconds=(timestamp_train_end - timestamp_train_start))))

Epoch  1  started...
Epoch [1/50], Step [100/396], Loss: 0.0171
=> Loss reduced by:	 8.982893377542496
   Saving model state
Epoch [1/50], Step [200/396], Loss: 0.0009
=> Loss reduced by:	 0.01619914174079895
   Saving model state
Epoch [1/50], Step [300/396], Loss: 0.0005
=> Loss reduced by:	 0.00038039684295654297
   Saving model state
Epoch done in  0:04:49.755447
Epoch  2  started...
Epoch [2/50], Step [100/396], Loss: 0.0009
Epoch [2/50], Step [200/396], Loss: 0.0006
Epoch [2/50], Step [300/396], Loss: 0.0002
=> Loss reduced by:	 0.000339701771736145
   Saving model state
Epoch done in  0:04:09.579911
Epoch  3  started...
Epoch [3/50], Step [100/396], Loss: 0.0106
Epoch [3/50], Step [200/396], Loss: 0.0009
Epoch [3/50], Step [300/396], Loss: 0.0005
Epoch done in  0:04:07.454669
Epoch  4  started...
Epoch [4/50], Step [100/396], Loss: 0.0000
=> Loss reduced by:	 0.00018502771854400635
   Saving model state
Epoch [4/50], Step [200/396], Loss: 0.0004
Epoch [4/50], Step [300/396], Los

In [15]:
#load the best model
model.load_state_dict(torch.load("state_dict.dct"))

model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
          
    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'final_state.dct')

Test Accuracy of the model on the test images: 99.49431099873578 %


In [16]:
timestamp_exec_end = time.time()
print("Total execution time: ",str(datetime.timedelta(seconds=(timestamp_exec_end - timestamp_exec_start))))

Total execution time:  3:27:53.668974
